In [13]:
from CBAStats.Player import *
from CBAStats.Team import *
from CBAStats.Player import stats_output
import datetime
from Scraping import Scraper
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
import pymysql

In [26]:
print('最近一次运行时间：',datetime.datetime.now())

最近一次运行时间： 2020-01-25 20:12:30.215883


# 载入数据

In [23]:
from sqlalchemy import create_engine
import pymysql
user_name = 'guest'
passcode = 'Guest123456'
endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
database = 'CBA_Data'
engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')

connection= engine.connect()

In [24]:
df = pd.read_sql("select * from CBA_Data.PlayerStatsPerGame", connection);

In [25]:
connection.close()

# 所有球队高阶数据统计

In [17]:
teams = Team('',df)

teams_df = pd.concat([teams.mov, teams.tm_pace, teams.tm_ortg, teams.tm_drtg, teams.tm_nrtg], axis=1)
teams_df.columns=['场均净胜分MOV', 'Pace', 'OffensiveRating', 'DefensiveRating', 'NetRating']
teams_df = teams_df.sort_values(by='NetRating',ascending=False)
teams_df

,场均净胜分MOV,Pace,OffensiveRating,DefensiveRating,NetRating
球队,,,,,
广东,16.966667,97.371742,127.174818,109.288114,17.886704
新疆,9.275862,95.685418,122.806200,113.006476,9.799723
青岛,6.551724,96.006708,121.484092,114.506469,6.977623
浙江,5.466667,92.180252,123.610183,117.591160,6.019023
山东,4.933333,91.976830,118.431880,112.950980,5.480900
辽宁,4.566667,94.518086,118.797868,113.871928,4.925941
广厦,4.533333,94.605367,113.978765,109.091907,4.886858
吉林,4.333333,92.575108,119.361926,114.580093,4.781834
山西,3.333333,91.331301,117.196559,113.471303,3.725256


In [11]:
teams_df.to_csv(teams_stats_output)

# 所有球员高阶数据

In a later chapter of Basketball on Paper, Oliver emphasized that Offensive Ratings shouldn't be viewed in a vacuum. Introducing a concept he called "Skill Curves", he acknowledged that a player's ORtg needed to be judged in conjunction with his Usage Rate, a measure of how big a role the player fills in his team's offense. The bigger the role, the more difficult it is to maintain a high ORtg; the smaller the role, the easier it is to be highly efficient. Because of this, Oliver stressed that a player's ORtg should primarily be compared to those of other players in a similar role.

Out of necessity (owing to a lack of defensive data in the basic boxscore), individual Defensive Ratings are heavily influenced by the team's defensive efficiency. They assume that all teammates are equally good (per minute) at forcing non-steal turnovers and non-block misses, as well as assuming that all teammates face the same number of total possessions per minute.

In [18]:
players = Player('',df)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
players_df = pd.concat([players.plr_ortg, players.plr_drtg, players.plr_nrtg,players.plr_usg], axis=1)

players_df.columns=['OffensiveRating', 'DefensiveRating', 'NetRating','UsagePercent']
players_df = players_df.sort_values(by=['NetRating','UsagePercent'],ascending=False)


players_df = pd.merge(players_df,players.plr_total_stats,left_index=True,right_index=True)
players_df.reset_index(inplace=True)
players_df.sort_values(by=['球队','NetRating','UsagePercent'],ascending=False,inplace=True)
players_df

,球员,球队,OffensiveRating,DefensiveRating,NetRating,UsagePercent,出场时间,首发,进攻篮板,防守篮板,...,扣篮,被侵,得分,2分中,2分投,3分中,3分投,罚球中,罚球投,出场
267,约翰逊,青岛,131.671544,118.351276,13.320269,27.961744,893,1,138,283,...,36,251,687,240,382,0,4,207,299,29
229,焦海龙,青岛,123.648987,120.205521,3.443466,15.806572,442,7,30,48,...,2,22,181,58,95,17,44,14,15,19
239,王庆明,青岛,122.544756,121.379809,1.164948,13.066812,777,22,15,68,...,1,50,265,37,75,54,127,29,36,29
38,刘传兴,青岛,120.945401,119.964395,0.981005,18.934261,313,18,51,75,...,7,45,141,53,92,0,0,35,57,26
12,亚当斯,青岛,115.163768,116.408521,-1.244753,39.500900,1036,28,27,209,...,1,223,1012,214,379,122,346,218,249,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,吴永盛,上海,81.548796,119.875392,-38.326595,18.329778,51,2,1,2,...,0,4,16,7,11,0,5,2,2,7
182,朱瀛,上海,NaN,126.901512,NaN,0.000000,2,0,1,0,...,0,0,0,0,0,0,0,0,0,1
81,周彦旭,上海,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,唐子豪,上海,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
players_df.to_csv(players_stats_output,index=False)

# 单个球队高阶数据统计

In [19]:
teams = Team('广东',df)

teams_df = pd.concat([teams.mov, teams.tm_pace, teams.tm_ortg, teams.tm_drtg, teams.tm_nrtg], axis=1)
teams_df.columns=['场均净胜分MOV', 'Pace', 'OffensiveRating', 'DefensiveRating', 'NetRating']
teams_df = teams_df.sort_values(by='NetRating',ascending=False)
teams_df

,场均净胜分MOV,Pace,OffensiveRating,DefensiveRating,NetRating
球队,,,,,
广东,16.966667,97.371742,127.174818,109.288114,17.886704


# 单个球员高阶数据

In [20]:
players = Player('易建联',df)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
players_df = pd.concat([players.plr_ortg, players.plr_drtg, players.plr_nrtg,players.plr_usg], axis=1)

players_df.columns=['OffensiveRating', 'DefensiveRating', 'NetRating','UsagePercent']
players_df = players_df.sort_values(by=['NetRating','UsagePercent'],ascending=False)

players_df = pd.merge(players_df,players.plr_total_stats,left_index=True,right_index=True)
players_df

,,OffensiveRating,DefensiveRating,NetRating,UsagePercent,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,...,扣篮,被侵,得分,2分中,2分投,3分中,3分投,罚球中,罚球投,出场
球员,球队,,,,,,,,,,,,,,,,,,,,,
易建联,广东,129.622295,113.100746,16.521549,23.730147,939,30,82,247,42,59,...,67,167,608,175,307,35,96,153,190,30
